<a href="https://colab.research.google.com/github/deepakri201/NLSTNatureSciData/blob/main/TechnicalValidation/consistencyChecks/NLSTSegvsNLSTSybil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Compare all series that overlap

later run comparison over all series, see which have bounding boxes and no seg, and vice versa


Deepa Krishnaswamy
Brigham and Women's Hospital
October 2025


# Parameterization

In [ ]:
#@title Enter your Project ID here
# initialize this variable with your Google Cloud Project ID!
project_name = "idc-external-018" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = project_name

!gcloud config set project $project_name

from google.colab import auth
auth.authenticate_user()

Updated property [core/project].


# Environment setup

In [ ]:
!pip install idc-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 78.1 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.3.2
    Uninstalling duckdb-1.3.2:
      Successfully uninstalled duckdb-1.3.2


In [ ]:
import os
import sys
import time

import numpy as np
import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

import json
from pathlib import Path

In [ ]:
import matplotlib.patches as patches

In [ ]:
from google.cloud import bigquery
from google.cloud import storage

In [ ]:
import datetime

In [ ]:
from idc_index import IDCClient

idc_client = IDCClient.client()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install pydicom
import pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.2 MB/s eta 0:00:00


# Find the series that have both a bounding box and a segmentation

In [ ]:
# Get the Referenced SeriesInstanceUIDs of the Sybil bounding boxes
client_bq = bigquery.Client(project=project_name)
query = f"""
  SELECT
    DISTINCT(ReferencedSeriesInstanceUID)
  FROM
    `idc-external-018.sybil_and_nlstseg.bbox_measurements`
"""
df_sybil = client_bq.query(query).to_dataframe()

# Get the Referenced SeriesInstanceUIDs of the NLSTSeg
query = f"""
  SELECT
    DISTINCT(segmented_SeriesInstanceUID)
  FROM
    `idc-external-018.sybil_and_nlstseg.segmentations`
"""
df_nlstseg = client_bq.query(query).to_dataframe()

# Get the intersection
SeriesInstanceUIDs_list = list(set(df_sybil['ReferencedSeriesInstanceUID'].values) & set(df_nlstseg['segmented_SeriesInstanceUID'].values))

# Check
print('Sybil number of series: ' + str(len(df_sybil)))
print('NLSTSeg number of series: ' + str(len(df_nlstseg)))
print('Number of intersecting series: ' + str(len(SeriesInstanceUIDs_list)))



Sybil number of series: 970
NLSTSeg number of series: 597
Number of intersecting series: 402


# Get the overlapping slices

In [ ]:
query = f"""
  SELECT
    *
  FROM
    `idc-external-018.sybil_and_nlstseg.bbox_measurements`
  WHERE
    ReferencedSeriesInstanceUID IN UNNEST(@series_list)
"""
job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ArrayQueryParameter("series_list", "STRING", SeriesInstanceUIDs_list)])
df_sybil_bbox = client_bq.query(query, job_config=job_config).to_dataframe()

In [ ]:
df_sybil_bbox.head()

,PatientID,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,ReferencedSeriesInstanceUID,trackingIdentifier,trackingUniqueIdentifier,finding,findingSite,ReferencedSOPInstanceUID,ConceptNameCodeSequence,GraphicType,x0,y0,x1,y1,x2,y2,x3,y3
0,100147,1.2.840.113654.2.55.31958452963320032523273261...,1.2.826.0.1.3680043.8.498.80137012981695130585...,1.2.826.0.1.3680043.8.498.25702787397259362445...,1.2.840.113654.2.55.15708941008648745210499888...,Bounding box #1,1.2.826.0.1.3680043.8.498.19732880118785808469...,"{'CodeValue': '52988006', 'CodingSchemeDesigna...","{'CodeValue': '39607008', 'CodingSchemeDesigna...",1.2.840.113654.2.55.79807002501189556016815093...,"[{'CodeValue': '111030', 'CodingSchemeDesignat...",POLYLINE,130.021896,312.039795,172.815903,312.039795,172.815903,347.701508,130.021896,347.701508
1,100147,1.2.840.113654.2.55.31958452963320032523273261...,1.2.826.0.1.3680043.8.498.80137012981695130585...,1.2.826.0.1.3680043.8.498.25702787397259362445...,1.2.840.113654.2.55.15708941008648745210499888...,Bounding box #2,1.2.826.0.1.3680043.8.498.27663554135755923869...,"{'CodeValue': '52988006', 'CodingSchemeDesigna...","{'CodeValue': '39607008', 'CodingSchemeDesigna...",1.2.840.113654.2.55.15052476845087007628485738...,"[{'CodeValue': '111030', 'CodingSchemeDesignat...",POLYLINE,130.000000,312.000000,171.000000,312.000000,171.000000,348.000000,130.000000,348.000000
2,100147,1.2.840.113654.2.55.31958452963320032523273261...,1.2.826.0.1.3680043.8.498.80137012981695130585...,1.2.826.0.1.3680043.8.498.25702787397259362445...,1.2.840.113654.2.55.15708941008648745210499888...,Bounding box #3,1.2.826.0.1.3680043.8.498.45257385294575042278...,"{'CodeValue': '52988006', 'CodingSchemeDesigna...","{'CodeValue': '39607008', 'CodingSchemeDesigna...",1.2.840.113654.2.55.32901662060934990548341662...,"[{'CodeValue': '111030', 'CodingSchemeDesignat...",POLYLINE,129.000000,313.000000,169.000000,313.000000,169.000000,349.000000,129.000000,349.000000
3,100147,1.2.840.113654.2.55.31958452963320032523273261...,1.2.826.0.1.3680043.8.498.80137012981695130585...,1.2.826.0.1.3680043.8.498.25702787397259362445...,1.2.840.113654.2.55.15708941008648745210499888...,Bounding box #4,1.2.826.0.1.3680043.8.498.31896297182924717311...,"{'CodeValue': '52988006', 'CodingSchemeDesigna...","{'CodeValue': '39607008', 'CodingSchemeDesigna...",1.2.840.113654.2.55.21729370900780746886375992...,"[{'CodeValue': '111030', 'CodingSchemeDesignat...",POLYLINE,129.000000,313.000000,167.000000,313.000000,167.000000,350.000000,129.000000,350.000000
4,100147,1.2.840.113654.2.55.31958452963320032523273261...,1.2.826.0.1.3680043.8.498.80137012981695130585...,1.2.826.0.1.3680043.8.498.25702787397259362445...,1.2.840.113654.2.55.15708941008648745210499888...,Bounding box #5,1.2.826.0.1.3680043.8.498.21363506884817323687...,"{'CodeValue': '52988006', 'CodingSchemeDesigna...","{'CodeValue': '39607008', 'CodingSchemeDesigna...",1.2.840.113654.2.55.19851369674160913596156868...,"[{'CodeValue': '111030', 'CodingSchemeDesignat...",POLYLINE,129.000000,314.000000,166.000000,314.000000,166.000000,351.000000,129.000000,351.000000


In [ ]:
# Get a list of the Referenced SOPInstanceUIDs of the Sybil bounding boxes

query = f"""
  SELECT
    dicom_all.PatientID,
    dicom_all.StudyInstanceUID,
    rss.SeriesInstanceUID,
    ris.ReferencedSOPInstanceUID
  FROM
    `idc-external-018.sybil_and_nlstseg.dicom_all` AS dicom_all
  CROSS JOIN
    UNNEST(dicom_all.ReferencedSeriesSequence) AS rss
  CROSS JOIN
    UNNEST(rss.ReferencedInstanceSequence) AS ris
  WHERE
    rss.SeriesInstanceUID IN UNNEST(@series_list)
"""
job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ArrayQueryParameter("series_list", "STRING", SeriesInstanceUIDs_list)])
df_nlstseg_seg = client_bq.query(query, job_config=job_config).to_dataframe()

# # Get the intersection
# SOPInstanceUIDs_list = list(set(df_sybil_bbox['ReferencedSOPInstanceUID'].values ) & set(df_nlstseg_seg['ReferencedSOPInstanceUID'].values))

# # Check
# print('Sybil number of referenced SOPInstanceUIDs: ' + str(len(df_sybil_bbox)))
# print('NLSTSeg number of referenced SOPInstanceUIDs: ' + str(len(df_nlstseg_seg)))
# print('Number of intersecting referenced SOPInstanceUIDs: ' + str(len(SOPInstanceUIDs_list)))

In [ ]:
df_nlstseg_seg.head()

,PatientID,StudyInstanceUID,SeriesInstanceUID,ReferencedSOPInstanceUID
0,102736,1.2.840.113654.2.55.23485103792480070136869664...,1.2.840.113654.2.55.77451854363369829573866150...,1.2.840.113654.2.55.22633428721175074221864018...
1,102736,1.2.840.113654.2.55.23485103792480070136869664...,1.2.840.113654.2.55.77451854363369829573866150...,1.2.840.113654.2.55.34019444235248186592925698...
2,120573,1.2.840.113654.2.55.60312645929603274507035783...,1.2.840.113654.2.55.32568708984853805318844556...,1.2.840.113654.2.55.22650922629996972611142371...
3,120573,1.2.840.113654.2.55.60312645929603274507035783...,1.2.840.113654.2.55.32568708984853805318844556...,1.2.840.113654.2.55.32515570644216935462720137...
4,114248,1.2.840.113654.2.55.14865377394025870860835319...,1.2.840.113654.2.55.12363323410784959119051552...,1.2.840.113654.2.55.29006022640692810061379893...


In [ ]:
len(df_nlstseg_seg)

4998

# Get the overlap in terms of actual pixel segmentation and bounding box - so we can flag scans that don't have complete tumor coverage.

In [ ]:
df_patient = df_nlstseg_seg.copy(deep=True)[['PatientID', 'StudyInstanceUID', 'SeriesInstanceUID']]
df_patient = df_patient.drop_duplicates(subset=['PatientID', 'StudyInstanceUID', 'SeriesInstanceUID'])
df_patient.head()

PatientID_list = df_patient['PatientID'].values
StudyInstanceUID_list = df_patient['StudyInstanceUID'].values
SeriesInstanceUID_list = df_patient['SeriesInstanceUID'].values

In [ ]:
# We add the gcs_url of the SEG file to this dataframe
gcs_url_seg = [os.path.join("gs://nlstseg_seg_and_sr/seg", f + "_SEG.dcm") for f in SeriesInstanceUID_list]
df_patient['gcs_url_seg'] = gcs_url_seg

In [ ]:
len(df_patient)

402

In [ ]:
df_patient.head()

,PatientID,StudyInstanceUID,SeriesInstanceUID,gcs_url_seg
0,102736,1.2.840.113654.2.55.23485103792480070136869664...,1.2.840.113654.2.55.77451854363369829573866150...,gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.5...
2,120573,1.2.840.113654.2.55.60312645929603274507035783...,1.2.840.113654.2.55.32568708984853805318844556...,gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.5...
4,114248,1.2.840.113654.2.55.14865377394025870860835319...,1.2.840.113654.2.55.12363323410784959119051552...,gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.5...
6,113623,1.2.840.113654.2.55.22482478647078478355738832...,1.2.840.113654.2.55.17412435624899643050868386...,gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.5...
9,102537,1.2.840.113654.2.55.17698629574705452327296269...,1.2.840.113654.2.55.17923185272132498184218647...,gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.5...


In [ ]:
# Download all the SEG files from the bucket for this list of series
seg_directory_bucket = "gs://nlstseg_seg_and_sr/seg"
seg_directory = "/content/seg"
if not os.path.isdir(seg_directory):
  os.mkdir(seg_directory)

# Create the text file to hold gsc_url
gcsurl_temp = df_patient["gcs_url_seg"]
gs_file_path = "gcs_paths.txt"
gcsurl_temp.to_csv(gs_file_path, header = False, index = False)

# !cat manifest.txt | gsutil -m cp -I .
!cat gcs_paths.txt | gsutil -m cp -I $seg_directory

Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.77451854363369829573866150401935945777_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.3256870898485380531884455642265281171_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.123633234107849591190515522342469572867_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.174124356248996430508683867152648202173_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.179231852721324981842186473145321629778_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.182315093935481624682845229846809224632_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.38995485391900019876570761037952058249_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.158364186499334911697140176182635356136_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.135314423026715332810549867523376949749_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.8

In [ ]:
# Single mask/bbox
def tumor_fraction_in_box(mask, bbox):
  """
  mask: 2D numpy array (0/1 or bool) for tumor
  bbox: (xmin, ymin, xmax, ymax) in pixel coordinates
        (x = cols, y = rows)
  Returns: fraction of tumor voxels inside the box
  """
  xmin, ymin, xmax, ymax = bbox

  # clip to image bounds
  ymin = max(0, ymin); ymax = min(mask.shape[0], ymax)
  xmin = max(0, xmin); xmax = min(mask.shape[1], xmax)

  # box mask
  box_mask = np.zeros_like(mask, dtype=bool)
  box_mask[ymin:ymax, xmin:xmax] = True

  # intersection
  tumor_inside = np.logical_and(mask.astype(bool), box_mask)
  fraction = tumor_inside.sum() / mask.sum() if mask.sum() > 0 else np.nan

  return fraction

# List of masks/bboxes
def tumor_fraction_in_boxes(masks, bboxes):
    """
    Compute overall fraction of tumor voxels inside all boxes.

    Parameters
    ----------
    masks : list of 2D numpy arrays (bool or 0/1)
        Each mask corresponds to one slice.
    bboxes : list of tuples (xmin, ymin, xmax, ymax)
        Each bbox corresponds to the same slice index in masks.

    Returns
    -------
    fraction : float
        Overall fraction of tumor voxels inside all boxes across slices.
        Returns np.nan if no tumor voxels are present at all.
    """
    total_tumor_voxels = 0
    total_tumor_inside = 0

    for mask, bbox in zip(masks, bboxes):
        if mask is None or bbox is None:
            continue

        xmin, ymin, xmax, ymax = bbox

        # Clip to image bounds
        ymin = max(0, ymin); ymax = min(mask.shape[0], ymax)
        xmin = max(0, xmin); xmax = min(mask.shape[1], xmax)

        if ymin >= ymax or xmin >= xmax:
            continue  # skip degenerate boxes

        # Create box mask and compute intersection
        box_mask = np.zeros_like(mask, dtype=bool)
        box_mask[ymin:ymax, xmin:xmax] = True

        tumor_inside = np.logical_and(mask.astype(bool), box_mask)
        total_tumor_inside += tumor_inside.sum()
        total_tumor_voxels += mask.sum()

    if total_tumor_voxels == 0:
        return np.nan

    return total_tumor_inside / total_tumor_voxels



In [ ]:
PatientID_list_temp = ["118672"]

In [ ]:
### Calculate the overlap on a per series basis ###


df_overlap_all = pd.DataFrame()
print('Num patients: ' + str(len(PatientID_list)))

### Over each patient ###
for index, patient in enumerate(PatientID_list,1):
  print('*** On index: ' + str(index) + ' ***')
  print(patient)
  StudyInstanceUIDs = list(set(df_patient[df_patient['PatientID']==patient]['StudyInstanceUID'].values))

  ### Over each study ###
  for study in StudyInstanceUIDs:
    series = list(set(df_patient[df_patient['StudyInstanceUID']==study]['SeriesInstanceUID'].values))

    ### Over each series ###
    for series in series:

      ### Get the appropriate SEG file ###
      seg_filename = os.path.join("/content/seg", series + "_SEG.dcm")

      ### Get the referencedSOPInstanceUIDs and the mask indices ###
      # We know which slices have an overlap - by the referencedSOPInstanceUID
      # Now we need to get the pixel data for those slices.
      seg = pydicom.dcmread(seg_filename)
      SeriesInstanceUID_seg = seg.SeriesInstanceUID
      # Get the list of SegmentNumber and SegmentLabel
      segment_number_original_list = [f.SegmentNumber for f in seg.SegmentSequence]
      segment_label_original_list = [f.SegmentLabel for f in seg.SegmentSequence]
      # Get the pixel array data
      mask_data = seg.pixel_array
      # Need to get the per frame function groups - to know the referencedSOPInstanceUIDs
      pffg = seg.PerFrameFunctionalGroupsSequence
      num_slices_in_seg = len(pffg)
      print('num_slices_in_seg: ' + str(num_slices_in_seg))
      pffg_sop_list = []
      segment_number_list = []
      segment_label_list = []
      error_der_image_sequence = 0
      for n in range(0,num_slices_in_seg):
        try:
          sop = pffg[n].DerivationImageSequence[0].SourceImageSequence[0].ReferencedSOPInstanceUID
          segment_number = pffg[n].SegmentIdentificationSequence[0].ReferencedSegmentNumber
          # get the index of where the segment_number is in the segment_number_original_list
          segment_number_index = segment_number_original_list.index(segment_number)
          segment_label = segment_label_original_list[segment_number_index]
        except:
          print('ERROR: cannot access derivation image sequence')
          error_der_image_sequence = -1
          continue
        pffg_sop_list.append(sop)
        segment_number_list.append(segment_number)
        segment_label_list.append(segment_label)
      # Now create a dataframe with the SOPInstanceUID and the slice index into mask
      df_seg = pd.DataFrame()
      df_seg['SOPInstanceUID'] = pffg_sop_list
      df_seg['mask_slice'] = np.arange(0,len(pffg_sop_list))
      df_seg['SegmentNumber'] = segment_number_list
      df_seg['SegmentLabel'] = segment_label_list
      df_seg['num_of_slices_with_seg'] = [num_slices_in_seg] * len(pffg_sop_list)

      ### Now get the referencedSOPInstanceUID for the sybil bbox
      df_sybil_bbox_temp = df_sybil_bbox[df_sybil_bbox['ReferencedSeriesInstanceUID']==series]
      SeriesInstanceUID_bbox = df_sybil_bbox_temp['SeriesInstanceUID'].values[0]

      ### Now only keep the referencedSOPInstanceUIDs that overlap ###
      df_overlap = df_seg.merge(df_sybil_bbox, left_on='SOPInstanceUID', right_on='ReferencedSOPInstanceUID')
      df_overlap = df_overlap.drop(columns=['SOPInstanceUID_y'])
      df_overlap = df_overlap.rename(columns={'SOPInstanceUID_x':'SOPInstanceUID'})
      print('num slices with bbox: ' + str(len(df_overlap)))

      ### For each slice, calculate the overlap ###
      SOPInstanceUID_list = df_overlap['ReferencedSOPInstanceUID'].values
      mask_slice_list = df_overlap['mask_slice'].values

      mask_list = []
      bbox_list = []
      num_of_slices_with_bbox_list = []
      if (error_der_image_sequence==0):
        for SOPInstanceUID,mask_slice in zip(SOPInstanceUID_list,mask_slice_list):
          df_temp = df_overlap[df_overlap['ReferencedSOPInstanceUID']==SOPInstanceUID]
          x0 = df_temp['x0'].values[0]; y0 = df_temp['y0'].values[0]
          x1 = df_temp['x1'].values[0]; y1 = df_temp['y1'].values[0]
          x2 = df_temp['x2'].values[0]; y2 = df_temp['y2'].values[0]
          x3 = df_temp['x3'].values[0]; y3 = df_temp['y3'].values[0]
          width = x1 - x0 # can change later
          height = y2 - y1 # can change later
          xmin = x0 # can change later
          ymin = y0 # can change later
          xmax = xmin + width
          ymax = ymin + height
          bbox = [np.floor(xmin).astype(np.int16),
                  np.floor(ymin).astype(np.int16),
                  np.ceil(xmax).astype(np.int16),
                  np.ceil(ymax).astype(np.int16)]
          mask_list.append(mask_data[mask_slice,:,:])
          bbox_list.append(bbox)
          # num_of_slices_with_bbox_list.append(len(set(SOPInstanceUID_list)))
        # Calculate over the mask_list and bbox_list
        metric = tumor_fraction_in_boxes(mask_list, bbox_list)
        # We only keep unique series - as we want a metric per series instead of per SOPInstanceUID.
        # We need to group over SeriesInstanceUID as we want a list of the SegmentDescriptions
        df_overlap = (
          df_overlap.groupby(["PatientID", "StudyInstanceUID", "ReferencedSeriesInstanceUID"], as_index=False)
            .agg({"SegmentLabel": lambda x: sorted(x.unique().tolist())})
        )
        # df_overlap = df_overlap.drop_duplicates(subset=['PatientID', 'SeriesInstanceUID'], keep='first')
        # df_overlap = df_overlap.drop('SOPInstanceUID', axis=1)
        # Add to dataframe
        df_overlap['overlap_metric'] = metric
        # Add the number of slices that have seg
        df_overlap['num_of_slices_with_seg'] = num_slices_in_seg
        # Add the number of slices that have a bbox
        # df_overlap['num_of_slices_with_bbox'] = num_of_slices_with_bbox_list
        df_overlap['num_of_slices_with_bbox'] = len(set(SOPInstanceUID_list))
        # Add the number of slices that have a bounding box and tumor segmentation
        df_overlap['number_of_slices_with_bbox_and_seg'] = len(mask_list)
        # Add in the SeriesInstanceUID_seg
        # Add in the SeriesInstanceUID_bbox
        df_overlap['SeriesInstanceUID_seg'] = len(df_overlap) * [SeriesInstanceUID_seg]
        df_overlap['SeriesInstanceUID_bbox'] = len(df_overlap) * [SeriesInstanceUID_bbox]

  # Add to overall dataframe
  if (error_der_image_sequence==0):
    df_overlap_all = pd.concat([df_overlap_all, df_overlap], axis=0)



# Add the ohif url
viewer_url = ["https://viewers-sandbox-gha-testing.web.app/viewer/?StudyInstanceUIDs="
              + f
              + "&gcp=projects/idc-external-018/locations/us-central1/datasets/sybil_and_nlstseg/dicomStores/sybil_and_nlstseg"
              for f in df_overlap_all['StudyInstanceUID'].values]
df_overlap_all['viewer_url'] = viewer_url

# Add in the series specific viewer_url
series_viewer_url_list = ["https://viewers-sandbox-gha-testing.web.app/viewer/?StudyInstanceUIDs=" +
                          str(study) +  "&SeriesInstanceUIDs=" + str(ref_series) + ',' + str(seg_series) + ',' + str(bbox_series) +
                            "&gcp=projects/idc-external-018/locations/us-central1/datasets/sybil_and_nlstseg/dicomStores/sybil_and_nlstseg"
                          for study,ref_series,seg_series,bbox_series in
                          zip(df_overlap_all['StudyInstanceUID'].values,
                              df_overlap_all['ReferencedSeriesInstanceUID'].values,
                              df_overlap_all['SeriesInstanceUID_seg'].values,
                              df_overlap_all['SeriesInstanceUID_bbox'].values)]
df_overlap_all['series_viewer_url'] = series_viewer_url_list

# Keep specific columns
df_overlap_all = df_overlap_all[['PatientID', 'StudyInstanceUID', 'ReferencedSeriesInstanceUID', 'SegmentLabel',
                                 'num_of_slices_with_bbox', 'num_of_slices_with_seg',
                                 'overlap_metric', 'number_of_slices_with_bbox_and_seg',
                                 'SeriesInstanceUID_seg', 'SeriesInstanceUID_bbox',
                                 'viewer_url', 'series_viewer_url']]
df_overlap_all = df_overlap_all.sort_values(by=['PatientID', 'StudyInstanceUID', 'ReferencedSeriesInstanceUID'])
# df_overlap_all.rename({'ReferencedSeriesInstanceUID': 'SeriesInstanceUID'}, inplace=True)

# Save
df_overlap_all.to_csv("/content/sybil_and_nlstseg_overlap_per_series.csv")



Num patients: 402
*** On index: 1 ***
102736
num_slices_in_seg: 2
num slices with bbox: 2
*** On index: 2 ***
120573
num_slices_in_seg: 2
num slices with bbox: 2
*** On index: 3 ***
114248
num_slices_in_seg: 2
num slices with bbox: 2
*** On index: 4 ***
113623
num_slices_in_seg: 3
num slices with bbox: 3
*** On index: 5 ***
102537
num_slices_in_seg: 3
num slices with bbox: 3
*** On index: 6 ***
102267
num_slices_in_seg: 3
num slices with bbox: 2
*** On index: 7 ***
100242
num_slices_in_seg: 3
num slices with bbox: 3
*** On index: 8 ***
134009
num_slices_in_seg: 3
num slices with bbox: 3
*** On index: 9 ***
129155
num_slices_in_seg: 3
num slices with bbox: 3
*** On index: 10 ***
124533
num_slices_in_seg: 3
num slices with bbox: 3
*** On index: 11 ***
124998
num_slices_in_seg: 3
num slices with bbox: 3
*** On index: 12 ***
107434
num_slices_in_seg: 3
num slices with bbox: 3
*** On index: 13 ***
106432
num_slices_in_seg: 3
num slices with bbox: 2
*** On index: 14 ***
110716
num_slices_in_

In [ ]:
df_overlap

,PatientID,StudyInstanceUID,ReferencedSeriesInstanceUID,SegmentLabel,overlap_metric,num_of_slices_with_bbox,number_of_slices_with_bbox_and_seg,SeriesInstanceUID_seg,SeriesInstanceUID_bbox
0,217780,1.3.6.1.4.1.14519.5.2.1.7009.9004.743323274081...,1.3.6.1.4.1.14519.5.2.1.7009.9004.269922574105...,[Lesion 1 - Tumor - Upper lobe of left lung],0.982765,33,33,1.2.276.0.7230010.3.1.3.481037312.602215.17612...,1.2.826.0.1.3680043.8.498.55382655415626102610...


In [ ]:
df_overlap

,PatientID,StudyInstanceUID,ReferencedSeriesInstanceUID,SegmentLabel,overlap_metric
0,102736,1.2.840.113654.2.55.23485103792480070136869664...,1.2.840.113654.2.55.77451854363369829573866150...,[Lesion 1 - Tumor - Upper lobe of left lung],1.0


In [ ]:
num_of_slices_with_bbox_list

[2, 2]

# OLD - Get the overlap in terms of actual pixel segmentation and bounding box

In [ ]:
df_patient = df_nlstseg_seg.copy(deep=True)[['PatientID', 'StudyInstanceUID', 'SeriesInstanceUID']]
df_patient = df_patient.drop_duplicates(subset=['PatientID', 'StudyInstanceUID', 'SeriesInstanceUID'])
df_patient.head()

PatientID_list = df_patient['PatientID'].values
StudyInstanceUID_list = df_patient['StudyInstanceUID'].values
SeriesInstanceUID_list = df_patient['SeriesInstanceUID'].values

In [ ]:
# We add the gcs_url of the SEG file to this dataframe
gcs_url_seg = [os.path.join("gs://nlstseg_seg_and_sr/seg", f + "_SEG.dcm") for f in SeriesInstanceUID_list]
df_patient['gcs_url_seg'] = gcs_url_seg

In [ ]:
len(df_patient)

402

In [ ]:
df_patient.head()

,PatientID,StudyInstanceUID,SeriesInstanceUID,gcs_url_seg
0,102736,1.2.840.113654.2.55.23485103792480070136869664...,1.2.840.113654.2.55.77451854363369829573866150...,gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.5...
2,120573,1.2.840.113654.2.55.60312645929603274507035783...,1.2.840.113654.2.55.32568708984853805318844556...,gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.5...
4,114248,1.2.840.113654.2.55.14865377394025870860835319...,1.2.840.113654.2.55.12363323410784959119051552...,gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.5...
6,113623,1.2.840.113654.2.55.22482478647078478355738832...,1.2.840.113654.2.55.17412435624899643050868386...,gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.5...
9,102537,1.2.840.113654.2.55.17698629574705452327296269...,1.2.840.113654.2.55.17923185272132498184218647...,gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.5...


In [ ]:
# Download all the SEG files from the bucket for this list of series
seg_directory_bucket = "gs://nlstseg_seg_and_sr/seg"
seg_directory = "/content/seg"
if not os.path.isdir(seg_directory):
  os.mkdir(seg_directory)

# Create the text file to hold gsc_url
gcsurl_temp = df_patient["gcs_url_seg"]
gs_file_path = "gcs_paths.txt"
gcsurl_temp.to_csv(gs_file_path, header = False, index = False)

# !cat manifest.txt | gsutil -m cp -I .
!cat gcs_paths.txt | gsutil -m cp -I $seg_directory

Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.77451854363369829573866150401935945777_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.3256870898485380531884455642265281171_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.123633234107849591190515522342469572867_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.174124356248996430508683867152648202173_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.179231852721324981842186473145321629778_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.182315093935481624682845229846809224632_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.158364186499334911697140176182635356136_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.38995485391900019876570761037952058249_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.333959368472243697306539144681085718022_SEG.dcm...
Copying gs://nlstseg_seg_and_sr/seg/1.2.8

In [ ]:
def tumor_fraction_in_box(mask, bbox):
  """
  mask: 2D numpy array (0/1 or bool) for tumor
  bbox: (xmin, ymin, xmax, ymax) in pixel coordinates
        (x = cols, y = rows)
  Returns: fraction of tumor voxels inside the box
  """
  xmin, ymin, xmax, ymax = bbox

  # clip to image bounds
  ymin = max(0, ymin); ymax = min(mask.shape[0], ymax)
  xmin = max(0, xmin); xmax = min(mask.shape[1], xmax)

  # box mask
  box_mask = np.zeros_like(mask, dtype=bool)
  box_mask[ymin:ymax, xmin:xmax] = True

  # intersection
  tumor_inside = np.logical_and(mask.astype(bool), box_mask)
  fraction = tumor_inside.sum() / mask.sum() if mask.sum() > 0 else np.nan

  return fraction


In [ ]:

df_overlap_all = pd.DataFrame()
print('Num patients: ' + str(len(PatientID_list)))

### Over each patient ###
for index, patient in enumerate(PatientID_list,1):
  print('*** On index: ' + str(index) + ' ***')
  print(patient)
  StudyInstanceUIDs = list(set(df_patient[df_patient['PatientID']==patient]['StudyInstanceUID'].values))

  ### Over each study ###
  for study in StudyInstanceUIDs:
    series = list(set(df_patient[df_patient['StudyInstanceUID']==study]['SeriesInstanceUID'].values))

    ### Over each series ###
    for series in series:

      ### Get the appropriate SEG file ###
      seg_filename = os.path.join("/content/seg", series + "_SEG.dcm")

      ### Get the referencedSOPInstanceUIDs and the mask indices ###
      # We know which slices have an overlap - by the referencedSOPInstanceUID
      # Now we need to get the pixel data for those slices.
      seg = pydicom.dcmread(seg_filename)
      # Get the pixel array data
      mask_data = seg.pixel_array
      # Need to get the per frame function groups - to know the referencedSOPInstanceUIDs
      pffg = seg.PerFrameFunctionalGroupsSequence
      num_slices_in_seg = len(pffg)
      print('num_slices_in_seg: ' + str(num_slices_in_seg))
      pffg_sop_list = []
      error_der_image_sequence = 0
      for n in range(0,num_slices_in_seg):
        try:
          sop = pffg[n].DerivationImageSequence[0].SourceImageSequence[0].ReferencedSOPInstanceUID
        except:
          print('ERROR: cannot access derivation image sequence')
          error_der_image_sequence = -1
          continue
        pffg_sop_list.append(sop)
      # Now create a dataframe with the SOPInstanceUID and the slice index into mask
      df_seg = pd.DataFrame()
      df_seg['SOPInstanceUID'] = pffg_sop_list
      df_seg['mask_slice'] = np.arange(0,len(pffg_sop_list))

      ### Now get the referencedSOPInstanceUID for the sybil bbox
      df_sybil_bbox_temp = df_sybil_bbox[df_sybil_bbox['SeriesInstanceUID']==series]

      ### Now only keep the referencedSOPInstanceUIDs that overlap ###
      df_overlap = df_seg.merge(df_sybil_bbox, left_on='SOPInstanceUID', right_on='ReferencedSOPInstanceUID')
      df_overlap = df_overlap.drop(columns=['SOPInstanceUID_y'])
      df_overlap = df_overlap.rename(columns={'SOPInstanceUID_x':'SOPInstanceUID'})

      ### For each slice, calculate the overlap ###
      SOPInstanceUID_list = df_overlap['ReferencedSOPInstanceUID'].values
      mask_slice_list = df_overlap['mask_slice'].values

      metric_list = []
      if (error_der_image_sequence==0):
        for SOPInstanceUID,mask_slice in zip(SOPInstanceUID_list,mask_slice_list):
          df_temp = df_overlap[df_overlap['ReferencedSOPInstanceUID']==SOPInstanceUID]
          x0 = df_temp['x0'].values[0]; y0 = df_temp['y0'].values[0]
          x1 = df_temp['x1'].values[0]; y1 = df_temp['y1'].values[0]
          x2 = df_temp['x2'].values[0]; y2 = df_temp['y2'].values[0]
          x3 = df_temp['x3'].values[0]; y3 = df_temp['y3'].values[0]
          width = x1 - x0 # can change later
          height = y2 - y1 # can change later
          xmin = x0 # can change later
          ymin = y0 # can change later
          xmax = xmin + width
          ymax = ymin + height
          bbox = [np.floor(xmin).astype(np.int16),
                  np.floor(ymin).astype(np.int16),
                  np.ceil(xmax).astype(np.int16),
                  np.ceil(ymax).astype(np.int16)]
          metric = tumor_fraction_in_box(mask_data[mask_slice,:,:], bbox)
          metric_list.append(metric)
        # Add to dataframe
        df_overlap['overlap_metric'] = metric_list

  # Add to overall dataframe
  if (error_der_image_sequence==0):
    df_overlap_all = pd.concat([df_overlap_all, df_overlap], axis=0)



# Add the ohif url
viewer_url = ["https://viewers-sandbox-gha-testing.web.app/viewer/?StudyInstanceUIDs=" + f + "&gcp=projects/idc-external-018/locations/us-central1/datasets/sybil_and_nlstseg/dicomStores/sybil_and_nlstseg" for f in df_overlap_all['StudyInstanceUID'].values]
df_overlap_all['viewer_url'] = viewer_url

# Save
df_overlap_all.to_csv("/content/sybil_and_nlstseg_overlap.csv")

Num patients: 402
*** On index: 1 ***
102736
num_slices_in_seg: 2
*** On index: 2 ***
120573
num_slices_in_seg: 2
*** On index: 3 ***
114248
num_slices_in_seg: 2
*** On index: 4 ***
113623
num_slices_in_seg: 3
*** On index: 5 ***
102537
num_slices_in_seg: 3
*** On index: 6 ***
102267
num_slices_in_seg: 3
*** On index: 7 ***
134009
num_slices_in_seg: 3
*** On index: 8 ***
100242
num_slices_in_seg: 3
*** On index: 9 ***
124533
num_slices_in_seg: 3
*** On index: 10 ***
129155
num_slices_in_seg: 3
*** On index: 11 ***
124998
num_slices_in_seg: 3
*** On index: 12 ***
107434
num_slices_in_seg: 3
*** On index: 13 ***
108352
num_slices_in_seg: 3
*** On index: 14 ***
110716
num_slices_in_seg: 3
*** On index: 15 ***
106432
num_slices_in_seg: 3
*** On index: 16 ***
106000
num_slices_in_seg: 3
*** On index: 17 ***
206732
num_slices_in_seg: 3
*** On index: 18 ***
216940
num_slices_in_seg: 3
*** On index: 19 ***
206687
num_slices_in_seg: 3
*** On index: 20 ***
205687
num_slices_in_seg: 3
*** On inde

In [ ]:
# 133076 - problem in accessing derivation image sequence - check later

# OLD - Get the overlap in terms of number of slices

In [ ]:
 # For each series that overlaps, get the:
 # 1. # of slices with bbox - done
 # 2. # of slices with seg - done
 # 3. # of slices that have both bbox and seg
 # 4. # of slices that have bbox and no seg
 # 5. # of slices that have seg and no bbox

# 1. number of slices with bbox
df_sybil_bbox_num_slices = df_sybil_bbox.groupby('ReferencedSeriesInstanceUID')['ReferencedSOPInstanceUID'].nunique()
df_sybil_num_boxes = df_sybil_bbox_num_slices.to_frame()
df_sybil_num_boxes = df_sybil_num_boxes.reset_index()
df_sybil_num_boxes = df_sybil_num_boxes.rename(columns={'ReferencedSOPInstanceUID':'sybil_num_boxes'})

# 2. number of slices with seg
df_nlstseg_seg_num_slices = df_nlstseg_seg.groupby('SeriesInstanceUID')['ReferencedSOPInstanceUID'].nunique()
df_nlstseg_num_slices = df_nlstseg_seg_num_slices.to_frame()
df_nlstseg_num_slices = df_nlstseg_num_slices.reset_index()
df_nlstseg_num_slices = df_nlstseg_num_slices.rename(columns={'SeriesInstanceUID': 'ReferencedSeriesInstanceUID',
                                                              'ReferencedSOPInstanceUID':'nlstseg_num_slices'})

# since we only care about the series that overlap for now, let's join these two
df_sybil_num_boxes['ReferencedSeriesInstanceUID'] = [str(f) for f in df_sybil_num_boxes['ReferencedSeriesInstanceUID'].values]
df_nlstseg_num_slices['ReferencedSeriesInstanceUID'] = [str(f) for f in df_nlstseg_num_slices['ReferencedSeriesInstanceUID'].values]
ReferencedSeriesInstanceUID_overlap = list(set(df_sybil_num_boxes['ReferencedSeriesInstanceUID'].values) & set(df_nlstseg_num_slices['ReferencedSeriesInstanceUID'].values))

# # keep these
# df_sybil_num_boxes = df_sybil_num_boxes[df_sybil_num_boxes['ReferencedSeriesInstanceUID'].isin(ReferencedSeriesInstanceUID_overlap)]
# df_nlstseg_num_slices = df_nlstseg_num_slices[df_nlstseg_num_slices['ReferencedSeriesInstanceUID'].isin(ReferencedSeriesInstanceUID_overlap)]
# # join
# df_counts = pd.merge(df_sybil_num_boxes, df_nlstseg_num_slices, how='inner', on='ReferencedSeriesInstanceUID')

# 3. number of slices with both bbox and seg
# 4. number of slices with bbox and no seg
# 5. number of slices with seg and no bbox
num_boxes_list = []
num_seg_list = []
overlap_list = []
only_bbox_list = []
only_seg_list = []
for series in ReferencedSeriesInstanceUID_overlap:
  bbox_sop = list(set(df_sybil_bbox[df_sybil_bbox['ReferencedSeriesInstanceUID']==series]['ReferencedSOPInstanceUID'].values))
  seg_sop = list(set(df_nlstseg_seg[df_nlstseg_seg['SeriesInstanceUID']==series]['ReferencedSOPInstanceUID'].values))
  num_boxes = len(list(set(bbox_sop)))
  num_seg = len(list(set(seg_sop)))
  overlap = len(list(set(bbox_sop) & set(seg_sop)))
  only_bbox = len(list(set(bbox_sop) - set(seg_sop)))
  only_seg = len(list(set(seg_sop) - set(bbox_sop)))
  # append
  num_boxes_list.append(num_boxes)
  num_seg_list.append(num_seg)
  overlap_list.append(overlap)
  only_bbox_list.append(only_bbox)
  only_seg_list.append(only_seg)

df_counts = pd.DataFrame()
df_counts['ReferencedSeriesInstanceUID'] = ReferencedSeriesInstanceUID_overlap
df_counts['num_bbox'] = num_boxes_list
df_counts['num_seg'] = num_seg_list
df_counts['overlap'] = overlap_list
df_counts['only_bbox'] = only_bbox_list
df_counts['only_seg'] = only_seg_list

# Calculate the overlap ratio, quick way to filter for low/high overlap:
# overlap_ratio = #overlap_slices / (#bbox_slices + #seg_slices - #overlap_slices)
df_counts['overlap_ratio'] = [f/(g+h-f) for f,g,h in zip(df_counts['overlap'].values, df_counts['num_bbox'].values, df_counts['num_seg'].values)]


In [ ]:
print(np.min(df_counts['overlap_ratio'].values))
print(np.max(df_counts['overlap_ratio'].values))

0.0
1.0


In [ ]:
# Add PatientID, StudyInstanceUID and viewer_url

PatientID_list = []
StudyInstanceUID_list = []
for index, row in df_counts.iterrows():
  series = row['ReferencedSeriesInstanceUID']
  PatientID = df_sybil_bbox[df_sybil_bbox["ReferencedSeriesInstanceUID"]==series]['PatientID'].values[0]
  StudyInstanceUID = df_sybil_bbox[df_sybil_bbox["ReferencedSeriesInstanceUID"]==series]['StudyInstanceUID'].values[0]
  PatientID_list.append(PatientID)
  StudyInstanceUID_list.append(StudyInstanceUID)
df_counts['PatientID'] = PatientID_list
df_counts['StudyInstanceUID'] = StudyInstanceUID_list

# df_sybil_bbox_unique = df_sybil_bbox.drop_duplicates(subset="ReferencedSeriesInstanceUID", keep='first')
# PatientID_list = df_sybil_bbox_unique[df_sybil_bbox_unique['ReferencedSeriesInstanceUID'].isin(ReferencedSeriesInstanceUID_overlap)]['PatientID'].values
# StudyInstanceUID_list = df_sybil_bbox_unique[df_sybil_bbox_unique['ReferencedSeriesInstanceUID'].isin(ReferencedSeriesInstanceUID_overlap)]['StudyInstanceUID'].values

viewer_url_list = ["https://viewers-sandbox-gha-testing.web.app/viewer/?StudyInstanceUIDs=" + f +
                    "&gcp=projects/idc-external-018/locations/us-central1/datasets/sybil_and_nlstseg/dicomStores/sybil_and_nlstseg"
                    for f in StudyInstanceUID_list]

df_counts['PatientID'] = PatientID_list
df_counts['StudyInstanceUID'] = StudyInstanceUID_list
df_counts['viewer_url'] = viewer_url_list

df_counts = df_counts[['PatientID', 'StudyInstanceUID', 'ReferencedSeriesInstanceUID',
                       'num_bbox', 'num_seg', 'overlap', 'only_bbox',	'only_seg', 'overlap_ratio',
                       'viewer_url']]
df_counts = df_counts.sort_values(by=['PatientID', 'StudyInstanceUID', 'ReferencedSeriesInstanceUID'])

In [ ]:
df_counts.head()

,PatientID,StudyInstanceUID,ReferencedSeriesInstanceUID,num_bbox,num_seg,overlap,only_bbox,only_seg,overlap_ratio,viewer_url
296,100147,1.2.840.113654.2.55.31958452963320032523273261...,1.2.840.113654.2.55.15708941008648745210499888...,8,8,8,0,0,1.000000,https://viewers-sandbox-gha-testing.web.app/vi...
166,100158,1.2.840.113654.2.55.81185422866512279860334872...,1.2.840.113654.2.55.31060976780967844152296392...,7,7,7,0,0,1.000000,https://viewers-sandbox-gha-testing.web.app/vi...
145,100242,1.2.840.113654.2.55.22835224307907880875083018...,1.2.840.113654.2.55.38995485391900019876570761...,7,3,3,4,0,0.428571,https://viewers-sandbox-gha-testing.web.app/vi...
334,100518,1.2.840.113654.2.55.36142141360017014809389003...,1.2.840.113654.2.55.81136962133262551156371928...,4,17,0,4,17,0.000000,https://viewers-sandbox-gha-testing.web.app/vi...
172,100570,1.2.840.113654.2.55.88862626250387223376057356...,1.2.840.113654.2.55.32380467633296345717423514...,8,10,7,1,3,0.636364,https://viewers-sandbox-gha-testing.web.app/vi...


In [ ]:
# save to csv, and then will save to BQ table

df_counts.to_csv("/content/sybil_and_nlstseg_overlap_slice_counts.csv")

# Old

In [ ]:
# Get the segmentation

In [ ]:
# Get a list of the Referenced SOPInstanceUIDs of the NLSTSeg
query = f"""
  SELECT
    *
  FROM
    `idc-external-018.sybil_and_nlstseg.segmentations`
  WHERE
    segmented_SeriesInstanceUID IN UNNEST(@series_list)
"""
job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ArrayQueryParameter("series_list", "STRING", SeriesInstanceUIDs_list)])
df_seg = client_bq.query(query, job_config=job_config).to_dataframe()

In [ ]:
# We have the per frame functional groups in dicom_all! So somehow have to query and get the ReferencedSOPInstanceUID list




In [ ]:
# We get the segmentation file from the bucket
seg_filename_bucket = os.path.join("gs://nlstseg_seg_and_sr/seg", series_pick + "_SEG.dcm")
seg_filename = os.path.join("/content/seg", series_pick + "_SEG.dcm")
if not os.path.isdir("/content/seg"):
  os.mkdir("/content/seg")
!gsutil cp $seg_filename_bucket $seg_filename

Copying gs://nlstseg_seg_and_sr/seg/1.2.840.113654.2.55.294795888751210912516507666864500510900_SEG.dcm...
/ [1 files][361.4 KiB/361.4 KiB]                                                
Operation completed over 1 objects/361.4 KiB.                                    


In [ ]:
# We know which slices have an overlap - by the referencedSOPInstanceUID
# Now we need to get the pixel data for those slices.

seg = pydicom.dcmread(seg_filename)

# Get the pixel array data
mask_data = seg.pixel_array
print(mask_data.shape)

# Need to get the per frame function groups - to know the referencedSOPInstanceUIDs
pffg = seg.PerFrameFunctionalGroupsSequence
num_slices_in_seg = len(pffg)
print('num_slices_in_seg: ' + str(num_slices_in_seg))
pffg_sop_list = []
for n in range(0,num_slices_in_seg):
  sop = pffg[n].DerivationImageSequence[0].SourceImageSequence[0].ReferencedSOPInstanceUID
  pffg_sop_list.append(sop)

# Now create a dataframe with the SOPInstanceUID and the slice index into mask
df_seg = pd.DataFrame()
df_seg['SOPInstanceUID'] = pffg_sop_list
df_seg['mask_slice'] = np.arange(0,len(pffg_sop_list))

(11, 512, 512)
num_slices_in_seg: 11


In [ ]:
df_seg

,SOPInstanceUID,mask_slice
0,1.2.840.113654.2.55.13998794245265636889281938...,0
1,1.2.840.113654.2.55.92014902492788269776983279...,1
2,1.2.840.113654.2.55.84705608724374715224219052...,2
3,1.2.840.113654.2.55.30065144618242790472755750...,3
4,1.2.840.113654.2.55.33864278056094434912780366...,4
5,1.2.840.113654.2.55.82622644398986097265317855...,5
6,1.2.840.113654.2.55.24017737918811694854352532...,6
7,1.2.840.113654.2.55.19755032706118208307243774...,7
8,1.2.840.113654.2.55.66323297372561385302687603...,8
9,1.2.840.113654.2.55.29148859922133669813638716...,9


In [ ]:
# Get the gcs_urls of the SOPInstanceUIDs_list - for plotting

query = f"""
  SELECT
    SOPInstanceUID,
    gcs_url
  FROM
    `bigquery-public-data.idc_current.dicom_all`
  WHERE
    SOPInstanceUID IN UNNEST(@SOPInstanceUIDs_list)
"""
job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ArrayQueryParameter("SOPInstanceUIDs_list", "STRING", SOPInstanceUIDs_list)])
df_sop_urls = client_bq.query(query, job_config=job_config).to_dataframe()


In [ ]:
df_sop_urls

,SOPInstanceUID,gcs_url
0,1.2.840.113654.2.55.66323297372561385302687603...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...
1,1.2.840.113654.2.55.29148859922133669813638716...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...
2,1.2.840.113654.2.55.19755032706118208307243774...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...
3,1.2.840.113654.2.55.82622644398986097265317855...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...
4,1.2.840.113654.2.55.24017737918811694854352532...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...
5,1.2.840.113654.2.55.84705608724374715224219052...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...
6,1.2.840.113654.2.55.30065144618242790472755750...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...
7,1.2.840.113654.2.55.33864278056094434912780366...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...
8,1.2.840.113654.2.55.92014902492788269776983279...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...
9,1.2.840.113654.2.55.13998794245265636889281938...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...


In [ ]:
# Now we join df_seg and df_sop_urls

df_plot = pd.merge(df_sop_urls, df_seg, left_on='SOPInstanceUID', right_on='SOPInstanceUID')

# Order by mask_slice
df_plot = df_plot.sort_values(by='mask_slice', ascending=True)

In [ ]:
df_plot

,SOPInstanceUID,gcs_url,mask_slice
9,1.2.840.113654.2.55.13998794245265636889281938...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...,0
8,1.2.840.113654.2.55.92014902492788269776983279...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...,1
5,1.2.840.113654.2.55.84705608724374715224219052...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...,2
6,1.2.840.113654.2.55.30065144618242790472755750...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...,3
7,1.2.840.113654.2.55.33864278056094434912780366...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...,4
3,1.2.840.113654.2.55.82622644398986097265317855...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...,5
4,1.2.840.113654.2.55.24017737918811694854352532...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...,6
2,1.2.840.113654.2.55.19755032706118208307243774...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...,7
0,1.2.840.113654.2.55.66323297372561385302687603...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...,8
1,1.2.840.113654.2.55.29148859922133669813638716...,gs://idc-open-data/2e0111bc-82e6-4285-8daa-cff...,9
